In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

In [2]:
%load_ext autoreload
%autoreload 1
%aimport utils_read_parsing
from utils_timeline_viz import *
from utils_read_parsing import *
import matplotlib.pyplot as plt
from tabulate import tabulate
from scipy.stats import mannwhitneyu
import seaborn as sns
import statsmodels.api as sm
pd.set_option("display.width", 100)
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.graphics.api import interaction_plot, abline_plot
from statsmodels.stats.anova import anova_lm
from statsmodels.discrete.discrete_model import Logit
%matplotlib inline
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 8, 6
pd.set_option('precision',3)
np.set_printoptions(precision=3,suppress=True)

d:\applications\anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
t = [['student','variable','pre','post','index','sim','CVS table','CVS graph']]
t.append(['s1','v1','1.0','2.333','1','L','Yes','No'])
t.append(['s1','v2','...','...','1','L','Yes','Yes'])
t.append(['s1','v3','...','...','2','C','No','No'])
t.append(['s1','v4','...','...','2','C','Yes','No'])
# print tabulate(t)

In [4]:
table_cvs_df = pd.read_csv('table_cvs_results.txt', sep='\t')
graph_cvs_df = pd.read_csv('graph_cvs_results.txt', sep='\t')

In [5]:
metadf = get_student_metadata()
order = dict(zip(metadf.index,metadf['activity order']))
graph_cvs_df['activity order'] = graph_cvs_df.studentid.apply(lambda sid: order[sid])
table_cvs_df['activity order'] = table_cvs_df.studentid.apply(lambda sid: order[sid])

In [6]:
variables = ["Area","Separation","Width","Concentration","Wavelength","Battery voltage"]

def binarize(number):
    if number>0 : return 1
    else: return 0

graph_cvs_df2 = graph_cvs_df.copy()
for v in variables:
    graph_cvs_df2[v] = graph_cvs_df.apply(lambda row: binarize(row[v]), axis=1)

table_cvs_df2 = table_cvs_df.copy()
for v in variables:
    table_cvs_df2[v] = table_cvs_df.apply(lambda row: binarize(row[v]), axis=1)

graph_cvs_df2['sum'] = graph_cvs_df2[variables].sum(axis=1)
table_cvs_df2['sum'] = table_cvs_df2[variables].sum(axis=1)

table_cvs_df2['sum caps'] = table_cvs_df2[['Area','Separation']].sum(axis=1)
table_cvs_df2['sum light'] = table_cvs_df2[["Width","Concentration"]].sum(axis=1)

graph_cvs_df2['sum caps'] = graph_cvs_df2[['Area','Separation']].sum(axis=1)
graph_cvs_df2['sum light'] = graph_cvs_df2[["Width","Concentration"]].sum(axis=1)
# table_cvs_df2.head()

In [7]:
table_intervals_df = pd.read_csv('table_intervals_results.txt', sep='\t')

In [8]:
%reload_ext utils_read_parsing
#grab worksheet data
worksheets = get_worksheet_metadata('beers')
pre = get_pre_worksheet(sim='beers')
main = get_main_worksheet(sim='beers')

ids = get_students_to_analyze_log_worksheets('beers')
pre= pre[pre['Student ID'].isin(worksheets[(worksheets['Type']=='p')&worksheets['Student ID'].isin(ids)]['other id'].values)]
main= main[main['Student ID'].isin(worksheets[(worksheets['Type']=='m')&worksheets['Student ID'].isin(ids)]['other id'].values)]

#ids in pre/post match "other id" in worksheet metdata so we need to assign the correct id in logs for each entry in pre/post
pre['sid'] = pre['Student ID'].apply(lambda row: worksheets.loc[worksheets[(worksheets['other id']==row)].index[0],'Student ID'])
main['sid'] = main['Student ID'].apply(lambda row: worksheets.loc[worksheets[worksheets['other id']==row].index[0],'Student ID'])

# print len(ids),len(pre),len(main)

In [9]:
## make desired table for beers
melted_pre = pd.melt(pre, id_vars=['sid'], value_vars=['Concentration','Width'], var_name='variable',value_name='pre')
melted_main = pd.melt(main, id_vars=['sid'], value_vars=['Concentration','Width'], var_name='variable',value_name='main')

L_scores = melted_pre.merge(melted_main, on=['sid','variable'], how='outer')
L_scores['sim'] = 'L'

L_scores['CVS_table'] = L_scores.apply(lambda row: table_cvs_df2.loc[table_cvs_df2[table_cvs_df2['studentid']==row['sid']].index[0],row['variable']],axis=1)
L_scores['CVS_graph'] = L_scores.apply(lambda row: graph_cvs_df2.loc[graph_cvs_df2[graph_cvs_df2['studentid']==row['sid']].index[0],row['variable']],axis=1)

L_scores['sim_index'] = L_scores.apply(lambda row: table_cvs_df2.loc[table_cvs_df2[table_cvs_df2['studentid']==row['sid']].index[0],'activity order'].index(row['sim'])+1,axis=1)

In [10]:
# L_scores.sort_values('sid').head()

In [11]:
%reload_ext utils_read_parsing
#grab worksheet data
worksheets = get_worksheet_metadata('caps')
pre = get_pre_worksheet(sim='caps')
main = get_main_worksheet(sim='caps')

ids = get_students_to_analyze_log_worksheets('caps')
pre= pre[pre['Student ID'].isin(worksheets[(worksheets['Type']=='p')&worksheets['Student ID'].isin(ids)]['other id'].values)]
main= main[main['Student ID'].isin(worksheets[(worksheets['Type']=='m')&worksheets['Student ID'].isin(ids)]['other id'].values)]

#ids in pre/post match "other id" in worksheet metdata so we need to assign the correct id in logs for each entry in pre/post
pre['sid'] = pre['Student ID'].apply(lambda row: worksheets.loc[worksheets[(worksheets['other id']==row)].index[0],'Student ID'])
main['sid'] = main['Student ID'].apply(lambda row: worksheets.loc[worksheets[worksheets['other id']==row].index[0],'Student ID'])

# print len(ids),len(pre),len(main)

In [12]:
## make desired table for caps
melted_pre = pd.melt(pre, id_vars=['sid'], value_vars=['Area','Separation'], var_name='variable',value_name='pre')
melted_main = pd.melt(main, id_vars=['sid'], value_vars=['Area','Separation'], var_name='variable',value_name='main')

C_scores = melted_pre.merge(melted_main, on=['sid','variable'], how='outer')
C_scores['sim'] = 'C'

C_scores['CVS_table'] = C_scores.apply(lambda row: table_cvs_df2.loc[table_cvs_df2[table_cvs_df2['studentid']==row['sid']].index[0],row['variable']],axis=1)
C_scores['CVS_graph'] = C_scores.apply(lambda row: graph_cvs_df2.loc[graph_cvs_df2[graph_cvs_df2['studentid']==row['sid']].index[0],row['variable']],axis=1)

C_scores['sim_index'] = C_scores.apply(lambda row: table_cvs_df2.loc[table_cvs_df2[table_cvs_df2['studentid']==row['sid']].index[0],'activity order'].index(row['sim'])+1,axis=1)

# This is what the data looks like

In [13]:
data = pd.concat([L_scores,C_scores])
data['intervals_in_table'] = data.apply(lambda row: table_intervals_df.loc[table_intervals_df[table_intervals_df['studentid']==row['sid']].index[0],row['variable']],axis=1)
data['CVS_table_only'] = data.apply(lambda row: row['CVS_table']*(1-row['CVS_graph']), axis = 1)
data = data.reset_index(drop=True)
data.sort_values('sid').head(12)

,sid,variable,pre,main,sim,CVS_table,CVS_graph,sim_index,intervals_in_table,CVS_table_only
0,10127163,Concentration,1.0,3.0,L,1,1,1,True,0
148,10127163,Width,1.0,3.0,L,1,1,1,False,0
296,10127163,Area,3.0,3.0,C,1,1,2,False,0
444,10127163,Separation,3.0,3.0,C,1,1,2,False,0
538,10192168,Separation,1.0,2.0,C,1,1,1,False,0
96,10192168,Concentration,1.0,3.0,L,1,1,2,False,0
390,10192168,Area,1.0,2.0,C,1,0,1,False,1
244,10192168,Width,1.0,0.0,L,1,0,2,True,1
1,10232160,Concentration,1.0,1.0,L,1,1,1,False,0
149,10232160,Width,1.0,1.0,L,0,0,1,False,0


In [14]:
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
 
def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov

def clean_summary(model):
    lines = model.summary().as_text().split('\n')
    lines = [l for l in lines if 'C(sid)' not in l]
    return '\n'.join(lines)

# Investigating models for learning and CVS

Let's compare these models:
* CVS graph by itself
* CVS table by itself
* CVS graph and table
* CVS graph and table_only
* CVS with three levels (graph, table, or neither)
* data without instances of CVS graph to compare CVS table and neither

Keep in mind that:
* sid = student id
* sim = Capacitance or Light absorbance
* sim_index = activity order (1 or 2)

C(\*) simply insures that the variable is considered as a categorical variable.

**IMPORTANT:** For each model, I show the table with the Anova results and then the linear model result. We get our p-value and effect size from the Anova table and we get our coefficient and confidence intervals from for that variable from the linear regressin results - not the p-value (right?)

Also, for categorical variables with 2+ levels, the Anova table tells you weither that variable matters, the linear regression tells you how each level behaves.

## CVS graph by itself

In [15]:
formula = 'main ~ C(CVS_graph) + variable + pre + C(sim_index)*C(sim) + C(sid)'
model = ols(formula, data).fit()
print "Model: ",formula
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 2 errors\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed all the rows for student ids):\n"
print clean_summary(model)

Model:  main ~ C(CVS_graph) + variable + pre + C(sim_index)*C(sim) + C(sid)

Anova table using type 2 errors

                      sum_sq     df       F     PR(>F)  eta_sq  omega_sq
C(CVS_graph)           4.218    1.0  15.193  1.122e-04   0.016     0.015
variable               2.351    3.0   2.823  3.847e-02   0.009     0.006
C(sim_index)           3.731    1.0  13.440  2.766e-04   0.014     0.013
C(sim)                 0.572    1.0   2.061  1.518e-01   0.002     0.001
C(sid)               136.601  147.0   3.347  1.688e-22   0.504     0.353
C(sim_index):C(sim)    0.964    1.0   3.473  6.307e-02   0.004     0.003
pre                    0.950    1.0   3.422  6.502e-02   0.004     0.002
Residual             121.600  438.0     NaN        NaN     NaN       NaN

Here is the linear model with coefficients and confidence intervals (removed all the rows for student ids):

                            OLS Regression Results                            
Dep. Variable:                   main   R-sq

## CVS table by itself

In [16]:
formula = 'main ~ C(CVS_table) + variable + pre + C(sim_index)*C(sim) + C(sid)'
model = ols(formula, data).fit()
print "Model: ",formula
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 2 errors\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed all the rows for student ids):\n"
print clean_summary(model)

Model:  main ~ C(CVS_table) + variable + pre + C(sim_index)*C(sim) + C(sid)

Anova table using type 2 errors

                      sum_sq     df       F     PR(>F)  eta_sq   omega_sq
C(CVS_table)           2.667    1.0   9.485  2.202e-03   0.010  8.516e-03
variable               2.865    3.0   3.397  1.785e-02   0.010  7.217e-03
C(sim_index)           4.885    1.0  17.373  3.703e-05   0.017  1.643e-02
C(sim)                 0.521    1.0   1.853  1.741e-01   0.002  8.562e-04
C(sid)               143.664  147.0   3.476  7.230e-24   0.513  3.653e-01
C(sim_index):C(sim)    1.341    1.0   4.768  2.953e-02   0.005  3.781e-03
pre                    0.777    1.0   2.765  9.709e-02   0.003  1.771e-03
Residual             123.151  438.0     NaN        NaN     NaN        NaN

Here is the linear model with coefficients and confidence intervals (removed all the rows for student ids):

                            OLS Regression Results                            
Dep. Variable:                   ma

## CVS graph and table

In [17]:
formula = 'main ~ C(CVS_graph) + C(CVS_table) + variable + pre + C(sim_index)*C(sim) + C(sid)'
model = ols(formula, data).fit()
print "Model: ",formula
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 2 errors\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed all the rows for student ids):\n"
print clean_summary(model)

Model:  main ~ C(CVS_graph) + C(CVS_table) + variable + pre + C(sim_index)*C(sim) + C(sid)

Anova table using type 2 errors

                      sum_sq     df       F     PR(>F)  eta_sq   omega_sq
C(CVS_graph)           1.847    1.0   6.654  1.022e-02   0.007  5.854e-03
C(CVS_table)           0.296    1.0   1.066  3.024e-01   0.001  6.848e-05
variable               2.349    3.0   2.821  3.860e-02   0.009  5.655e-03
C(sim_index)           3.753    1.0  13.519  2.655e-04   0.014  1.296e-02
C(sim)                 0.533    1.0   1.921  1.664e-01   0.002  9.540e-04
C(sid)               135.939  147.0   3.331  2.601e-22   0.508  3.549e-01
C(sim_index):C(sim)    0.972    1.0   3.502  6.195e-02   0.004  2.591e-03
pre                    0.819    1.0   2.951  8.656e-02   0.003  2.020e-03
Residual             121.304  437.0     NaN        NaN     NaN        NaN

Here is the linear model with coefficients and confidence intervals (removed all the rows for student ids):

                         

## CVS graph and table_only

In [18]:
formula = 'main ~ C(CVS_graph) + C(CVS_table_only) + variable + pre + C(sim_index)*C(sim) + C(sid)'
model = ols(formula, data).fit()
print "Model: ",formula
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 2 errors\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed all the rows for student ids):\n"
print clean_summary(model)

Model:  main ~ C(CVS_graph) + C(CVS_table_only) + variable + pre + C(sim_index)*C(sim) + C(sid)

Anova table using type 2 errors

                      sum_sq     df       F     PR(>F)  eta_sq   omega_sq
C(CVS_graph)           4.365    1.0  15.725  8.555e-05   0.016  1.510e-02
C(CVS_table_only)      0.296    1.0   1.066  3.024e-01   0.001  6.785e-05
variable               2.349    3.0   2.821  3.860e-02   0.009  5.602e-03
C(sim_index)           3.753    1.0  13.519  2.655e-04   0.014  1.284e-02
C(sim)                 0.533    1.0   1.921  1.664e-01   0.002  9.451e-04
C(sid)               135.939  147.0   3.331  2.601e-22   0.503  3.516e-01
C(sim_index):C(sim)    0.972    1.0   3.502  6.195e-02   0.004  2.567e-03
pre                    0.819    1.0   2.951  8.656e-02   0.003  2.001e-03
Residual             121.304  437.0     NaN        NaN     NaN        NaN

Here is the linear model with coefficients and confidence intervals (removed all the rows for student ids):

                    

## CVS with three levels (graph=2, table=1, none=0)

We add a "CVS_levels" column to our data

In [19]:
data['CVS_levels']=0
data['CVS_levels']=data['CVS_table']+data['CVS_graph']
# print data[(data['CVS_table']==0)&(data['CVS_graph']==1)].shape
data.head()

,sid,variable,pre,main,sim,CVS_table,CVS_graph,sim_index,intervals_in_table,CVS_table_only,CVS_levels
0,10127163,Concentration,1.0,3.0,L,1,1,1,True,0,2
1,10232160,Concentration,1.0,1.0,L,1,1,1,False,0,2
2,10537160,Concentration,1.0,2.0,L,1,0,2,False,1,1
3,10375163,Concentration,0.0,2.0,L,1,0,2,False,1,1
4,10420167,Concentration,1.0,3.0,L,1,1,2,False,0,2


In [20]:
formula = 'main ~ C(CVS_levels) + variable + pre + C(sim_index)*C(sim) + C(sid)'
model = ols(formula, data).fit()
print "Model: ",formula
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 2 errors\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed all the rows for student ids):\n"
print clean_summary(model)

Model:  main ~ C(CVS_levels) + variable + pre + C(sim_index)*C(sim) + C(sid)

Anova table using type 2 errors

                      sum_sq     df       F     PR(>F)  eta_sq   omega_sq
C(CVS_levels)          4.514    2.0   8.131  3.411e-04   0.017  1.464e-02
variable               2.349    3.0   2.821  3.860e-02   0.009  5.605e-03
C(sim_index)           3.753    1.0  13.519  2.655e-04   0.014  1.285e-02
C(sim)                 0.533    1.0   1.921  1.664e-01   0.002  9.457e-04
C(sid)               135.939  147.0   3.331  2.601e-22   0.503  3.517e-01
C(sim_index):C(sim)    0.972    1.0   3.502  6.195e-02   0.004  2.568e-03
pre                    0.819    1.0   2.951  8.656e-02   0.003  2.002e-03
Residual             121.304  437.0     NaN        NaN     NaN        NaN

Here is the linear model with coefficients and confidence intervals (removed all the rows for student ids):

                            OLS Regression Results                            
Dep. Variable:                   m

## CVS table versus none

First, we keep only the instances when a student didn't do cvs_graph and use the "CVS_levels" variable again.

In [21]:
table_data = data[data['CVS_levels']<2]
print "We effectively reduce the number of data points from {0} to {1}".format(data.shape[0], table_data.shape[0])
print "We effectively reduce the number of students we analyze from {0} to {1}".format(len(set(list(data['sid']))), len(set(list(table_data['sid']))))
table_data.head()

We effectively reduce the number of data points from 592 to 301
We effectively reduce the number of students we analyze from 148 to 105


,sid,variable,pre,main,sim,CVS_table,CVS_graph,sim_index,intervals_in_table,CVS_table_only,CVS_levels
2,10537160,Concentration,1.0,2.0,L,1,0,2,False,1,1
3,10375163,Concentration,0.0,2.0,L,1,0,2,False,1,1
9,11095146,Concentration,1.0,2.0,L,0,0,2,False,0,0
11,11384795,Concentration,2.0,2.0,L,1,0,2,True,1,1
15,11892169,Concentration,0.0,2.0,L,0,0,1,False,0,0


In [22]:
formula = 'main ~ C(CVS_levels) + variable + pre + C(sim_index)*C(sim) + C(sid)'
model = ols(formula, table_data).fit()
print "Model: ",formula
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 2 errors\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed all the rows for student ids):\n"
print clean_summary(model)

Model:  main ~ C(CVS_levels) + variable + pre + C(sim_index)*C(sim) + C(sid)

Anova table using type 2 errors

                     sum_sq     df      F     PR(>F)     eta_sq   omega_sq
C(CVS_levels)         0.300    1.0  1.235  2.678e-01  2.134e-03  4.054e-04
variable              2.709    3.0  3.713  1.255e-02  1.924e-02  1.404e-02
C(sim_index)          1.289    1.0  5.301  2.239e-02  9.159e-03  7.418e-03
C(sim)                0.342    1.0  1.407  2.370e-01  2.432e-03  7.028e-04
C(sid)               87.821  104.0  3.472  5.110e-14  6.238e-01  4.434e-01
C(sim_index):C(sim)   2.094    1.0  8.610  3.755e-03  1.487e-02  1.312e-02
pre                   0.016    1.0  0.064  8.008e-01  1.103e-04 -1.615e-03
Residual             46.213  190.0    NaN        NaN        NaN        NaN

Here is the linear model with coefficients and confidence intervals (removed all the rows for student ids):

                            OLS Regression Results                            
Dep. Variable:           

CVS table is still not significant.

# Let's redo the chosen levels models with all two way interactions we might care about between control variables
We have 4 control variables:
* sim_index
* sim
* variable
* pre

We look at all interactions except the sim\*variable interaction (obviously...):
* C(sim_index)\*C(sim)
* **C(sim_index)\*variable** (prefer over *sim)
* C(sim_index)\*pre (not really interesting...)
* C(sim)\*pre
* **pre\*variable** (prefer over *sim)
* **variable*CVS_levels**
* **pre*CVS_levels**

## CVS levels

In [21]:
formula = 'main ~ C(CVS_levels)*variable + C(CVS_levels)*pre + C(sim_index)*variable + pre*variable + C(sid) + C(sim)'
model = ols(formula, data).fit()
print "Model: ",formula
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 2 errors\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed all the rows for student ids):\n"
print clean_summary(model)

Model:  main ~ C(CVS_levels)*variable + C(CVS_levels)*pre + C(sim_index)*variable + pre*variable + C(sid) + C(sim)

Anova table using type 2 errors

                         sum_sq     df       F     PR(>F)     eta_sq   omega_sq
C(CVS_levels)             4.390    2.0   7.825  4.599e-04  1.681e-02  1.464e-02
variable                  3.026    3.0   3.596  1.367e-02  1.159e-02  8.356e-03
C(sim_index)              3.418    1.0  12.186  5.319e-04  1.309e-02  1.200e-02
C(sid)                  126.112  147.0   3.059  3.598e-19  4.828e-01  3.246e-01
C(sim)                    1.313    1.0   4.680  3.108e-02  5.025e-03  3.947e-03
C(CVS_levels):variable    1.646    6.0   0.978  4.397e-01  6.301e-03 -1.423e-04
C(sim_index):variable     1.042    3.0   1.239  2.952e-01  3.991e-03  7.681e-04
pre                       0.725    1.0   2.586  1.086e-01  2.777e-03  1.701e-03
C(CVS_levels):pre         0.478    2.0   0.851  4.275e-01  1.829e-03 -3.188e-04
pre:variable              0.112    3.0   0.134  9.4

None of the interactions are significant...

# What model to use to look at CVS and incoming knowledge? (for CVS graph only)
## with variable interaction

Note that including "C(sid)" creates an error "Singular matrix". Maybe let's redo this analysis in SPSS.

In [25]:
formula = 'CVS_graph ~ pre*C(variable) + C(sim_index)*C(sim)'
print 'model: ', formula,'\n'
from patsy import dmatrices
Y, X = dmatrices(formula, data, return_type = 'dataframe')
# print X.columns
logit = Logit(Y, X)
model = logit.fit()
print model.summary()
# print model.params

model:  CVS_graph ~ pre*C(variable) + C(sim_index)*C(sim) 

Optimization terminated successfully.
         Current function value: 0.671971
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              CVS_graph   No. Observations:                  592
Model:                          Logit   Df Residuals:                      582
Method:                           MLE   Df Model:                            9
Date:                Thu, 18 Jan 2018   Pseudo R-squ.:                 0.03035
Time:                        11:15:43   Log-Likelihood:                -397.81
converged:                       True   LL-Null:                       -410.26
                                        LLR p-value:                  0.003079
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Inte

CVS plotting occurs more in second sim but no interaction with sim type. Pre doesn't matter, variable doesn't matter.

## no interaction

In [26]:
formula = 'CVS_graph ~ pre + C(variable) + C(sim_index)*C(sim)'
print 'model: ', formula,'\n'
from patsy import dmatrices
Y, X = dmatrices(formula, data, return_type = 'dataframe')
# print X.columns
logit = Logit(Y, X)
model = logit.fit()
print model.summary()
# print model.params

model:  CVS_graph ~ pre + C(variable) + C(sim_index)*C(sim) 

Optimization terminated successfully.
         Current function value: 0.678870
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:              CVS_graph   No. Observations:                  592
Model:                          Logit   Df Residuals:                      585
Method:                           MLE   Df Model:                            6
Date:                Thu, 18 Jan 2018   Pseudo R-squ.:                 0.02040
Time:                        11:15:43   Log-Likelihood:                -401.89
converged:                       True   LL-Null:                       -410.26
                                        LLR p-value:                   0.01031
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercep

pre matters. WHY???

## With all pair wise interactions

In [27]:
formula = 'CVS_graph ~ C(sim_index)*C(sim) + C(sim_index)*variable + C(sim_index)*pre + C(sim)*pre + pre*variable'
print 'model: ', formula,'\n'
from patsy import dmatrices
Y, X = dmatrices(formula, data, return_type = 'dataframe')
# print X.columns
logit = Logit(Y, X)
model = logit.fit()
print model.summary()
# print model.params

model:  CVS_graph ~ C(sim_index)*C(sim) + C(sim_index)*variable + C(sim_index)*pre + C(sim)*pre + pre*variable 

Optimization terminated successfully.
         Current function value: 0.670840
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              CVS_graph   No. Observations:                  592
Model:                          Logit   Df Residuals:                      579
Method:                           MLE   Df Model:                           12
Date:                Thu, 18 Jan 2018   Pseudo R-squ.:                 0.03198
Time:                        11:15:43   Log-Likelihood:                -397.14
converged:                       True   LL-Null:                       -410.26
                                        LLR p-value:                  0.009915
                                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------

d:\applications\anaconda2\lib\site-packages\statsmodels\base\model.py:1029: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
d:\applications\anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
d:\applications\anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
d:\applications\anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Lots of errors here. Not sure why... Either way, nothing is significant.